In [1]:
!pip install  pandas==1.4.0 numpy==1.20.0
!pip install keras_self_attention nbdev torchaudio==0.10.1  pyyaml h5py nvidia_smi keras==2.9.0rc1 
!pip install tensorflow librosa matplotlib pickle5 Pillow opencv-python numba 
!pip install tensorflow_addons librosa

     |████████████████████████████████| 11.7 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 41.9 MB/s eta 0:00:01    |██████████▋                     | 5.1 MB 41.9 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 47.4 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     |████████████████████████████████| 50 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 45.1 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 6.4 MB/s  eta 0:00:01
     |████████████████████████████████| 55 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 881.9 MB 25 kB/s s eta 0:00:013    |██████████████████▉             | 517.6 MB

In [2]:
#default_exp master

In [1]:
#export
from tensorflow.tools.docs import doc_controls
import librosa, librosa.display
import matplotlib.pyplot as plt

import os
import glob

import cv2
import pickle5 as pickle
from PIL import Image as im
import argparse
import math
import sys
import time
import copy
import numpy as np

import keras
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization #, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import models_custom 
from keras.models import load_model
import glob

import os

import pandas as pd
from tensorflow.tools.docs import doc_controls
import librosa, librosa.display
import matplotlib.pyplot as plt

import os
import glob

import cv2
import pickle5 as pickle
from PIL import Image as im
import argparse
import math
import sys
import time
import copy
import numpy as np

import keras
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization #, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import models_custom as mcs
from keras.models import load_model
import glob

import os

import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
therapist='none'
emotion='none'
t=30
feat=''
img=0
import importlib
importlib.reload(models_custom)
pathG='../data'
df = models_custom.create_label(therapist,emotion, pathG,balanced=0)
df = df.reset_index()
df['name2'] =df['name'].apply (lambda x: x.split(".")[0]+".pickle")
df = df.reset_index() # This redundant did that to avoid some errors

In [3]:
print(df.shape)


(1044, 10)


In [5]:
df.head()

,level_0,index,Unnamed: 0,therapist,name,emotion_type,rating,Rating,emotion,name2
0,0,16705,20000,none,1940_39117_1161625039.wav,none,none,1,0,1940_39117_1161625039.pickle
1,1,16706,20001,none,6529_53113_3443309365.wav,none,none,1,0,6529_53113_3443309365.pickle
2,2,16707,20002,none,9926_39117_1370960430.wav,none,none,1,0,9926_39117_1370960430.pickle
3,3,16708,20003,none,4205_39117_2156262111.wav,none,none,1,0,4205_39117_2156262111.pickle
4,4,16709,20004,none,65308_26501_3048138818.wav,none,none,1,0,65308_26501_3048138818.pickle


In [ ]:
#export
# Create mfcc layers 5000
from os.path import exists
import importlib
chunks=models_custom.split_dataframe(df,1)
wvs = { 'delta0':0,'delta1':1,'delta2':2 }
DATASET_PATHS=[]
for wv in wvs:
    DATASET_PATHS.append(pathG+"/mfcc/"+wv)
i=0

l95=5000 # Do not increase more than 5000
def cutpick(chunk):
    try:        
        name=chunk.iloc[0,9]
        fl = pathG+'/w2v2/three_mfcc/'+name
        if exists(fl)==False:
            height=0
            width=0
            A, B =models_custom.imgconvert(chunk ,l95,DATASET_PATHS,height, width, 0)
            print(fl)
            with open(fl, 'wb') as handle:            
                    pickle.dump(A, handle, protocol=pickle.HIGHEST_PROTOCOL)
            A=None
            B=None
    except Exception as e:
        print(str(e))
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1,backend='multiprocessing')(delayed(cutpick)(chunk) for chunk in chunks)

In [ ]:
#export
with open(pathG+'/duration.pickle', 'rb') as f:
        dur=pickle.load(f)
a=0
b=0
t=100
rmlst=[]
import os

for k in dur:    
    if dur[k] >t:
        rmlst.append(k)
        try:            
            os.remove(pathG+'/w2v2/three/'+k.split(".")[0]+".pickle")            
            a=a+1
        except:
            #print(pathG+'/w2v2/three/'+k.split(".")[0]+".pickle")
            b=b+1
        
df= df[~df['name'].isin(rmlst)]
print(df.shape)
print(a, b)
# Create layers 75 secs
from os.path import exists
import importlib
chunks=models_custom.split_dataframe(df,1)
wvs = { 'w2v2L3':3,'w2v2L6':6,'w2v2L8':8 }
DATASET_PATHS=[]
for wv in wvs:
    DATASET_PATHS.append(pathG+"/w2v2/"+wv)
i=0

l95=50*30 # Do not increase more than 5000
def cutpick(chunk):
    try:
        #print(chunk.iloc[0,9])

        name=chunk.iloc[0,9]
        fl = pathG+'/w2v2/three/'+name
        if 1==1: #exists(fl)==False:
            height=0
            width=0
            A, B =models_custom.imgconvert(chunk ,l95,DATASET_PATHS,height, width, 0)
            print(fl)
            with open(fl, 'wb') as handle:            
                    pickle.dump(A, handle, protocol=pickle.HIGHEST_PROTOCOL)
            A=None
            B=None
    except Exception as e:
        print(str(e))
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1,backend='multiprocessing')(delayed(cutpick)(chunk) for chunk in chunks)

In [ ]:
#export
# Create layers 100 secs
from os.path import exists
import importlib
chunks=models_custom.split_dataframe(df,1)
wvs = {'w2v2L0':0,'w2v2L1':1,'w2v2L2':2, 'w2v2L3':3,'w2v2L4':4,'w2v2L5':5,'w2v2L6':6,'w2v2L7':7,'w2v2L8':8,'w2v2L9':9,'w2v2L10':10,'w2v2L11':11 }
DATASET_PATHS=[]
for wv in wvs:
    DATASET_PATHS.append(pathG+"/w2v2/"+wv)
i=0
t=75
l95=5000# Do not increase more than 5000
def cutpick(chunk):
    try:
        #print(chunk.iloc[0,9])

        name=chunk.iloc[0,9]
        fl = pathG+'/w2v2/all/'+name
        if exists(fl)==False:
            height=0
            width=0
            A, B =models_custom.imgconvert(chunk ,l95,DATASET_PATHS,height, width, 0)
            print(fl)
            with open(fl, 'wb') as handle:            
                    pickle.dump(A, handle, protocol=pickle.HIGHEST_PROTOCOL)
            A=None
            B=None
    except Exception as e:
        print(str(e))
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1,backend='multiprocessing')(delayed(cutpick)(chunk) for chunk in chunks)

In [ ]:
#export
# Create img style 1/10 times scalling
from os.path import exists
import importlib
importlib.reload(models_custom)
chunks=models_custom.split_dataframe(df,1)
wvs = {'w2v2L0':0,'w2v2L1':1,'w2v2L2':2, 'w2v2L3':3,'w2v2L4':4,'w2v2L5':5,'w2v2L6':6,'w2v2L7':7,'w2v2L8':8,'w2v2L9':9,'w2v2L10':10,'w2v2L11':11 }
#wvs = {'w2v2L6':6}
DATASET_PATHS=[]
for wv in wvs:
    DATASET_PATHS.append(pathG+"/w2v2/"+wv)
    print(wv)
i=0
height=770
width=5000
def cutpick(chunk):
    try:
        #print(chunk.iloc[0,9])
        l95=5000 # Do not increase more than 5000
        name=chunk.iloc[0,9]
        fl = pathG+'/w2v2/imgs10/'+name
        if 1==1: #exists(fl)==False:
            A, B =models_custom.imgconvert(chunk ,l95,DATASET_PATHS,height, width, 1,10)
            #print(fl)
            with open(fl, 'wb') as handle:            
                    pickle.dump(A, handle, protocol=pickle.HIGHEST_PROTOCOL)
            A=None
            B=None
    except Exception as e:
        print(str(e))
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1,backend='multiprocessing')(delayed(cutpick)(chunk) for chunk in chunks)

In [ ]:
# Using only stats
with open(pathG+'/duration.pickle', 'rb') as f:
        dur=pickle.load(f)
l95=5000 #int(np.max(list(dur.values()))*50)
from os.path import exists
import importlib
chunks=models_custom.split_dataframe(df,1)
wvs = {'w2v2L0':0,'w2v2L1':1,'w2v2L2':2, 'w2v2L3':3,'w2v2L4':4,'w2v2L5':5,'w2v2L6':6,'w2v2L7':7,'w2v2L8':8,'w2v2L9':9,'w2v2L10':10,'w2v2L11':11 }
DATASET_PATHS=[]
for wv in wvs:
    DATASET_PATHS.append(pathG+"/w2v2/"+wv)
i=0
height=0
width=0
def cutpick(chunk):
    try:
        name=chunk.iloc[0,9]
        fl = pathG+'/w2v2/stats/'+name
        if exists(fl)==False:
            print(fl)
            A, B =models_custom.imgconvert(chunk ,l95,DATASET_PATHS,height, width, 0, scaleImg=-1)
            print(fl)
            with open(fl, 'wb') as handle:            
                    pickle.dump(A, handle, protocol=pickle.HIGHEST_PROTOCOL)
            A=None
            B=None
    except Exception as e:
        print(str(e))
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1,backend='multiprocessing')(delayed(cutpick)(chunk) for chunk in chunks)

In [ ]:
# Using only stats
import importlib
importlib.reload(models_custom)

with open(pathG+'/duration.pickle', 'rb') as f:
        dur=pickle.load(f)
l95=5003 #int(np.max(list(dur.values()))*50)
from os.path import exists
import importlib
chunks=models_custom.split_dataframe(df,1)
wvs = {'w2v2L0':0,'w2v2L1':1,'w2v2L2':2, 'w2v2L3':3,'w2v2L4':4,'w2v2L5':5,'w2v2L6':6,'w2v2L7':7,'w2v2L8':8,'w2v2L9':9,'w2v2L10':10,'w2v2L11':11 }
wvs = {'w2v2L6':6}
DATASET_PATHS=[]
for wv in wvs:
    DATASET_PATHS.append(pathG+"/w2v2/"+wv)
i=0
height=0
width=0
def cutpick(chunk):
    try:
        name=chunk.iloc[0,9]
        fl = pathG+'/w2v2/statsT/'+name
        if exists(fl)==False:
            print(fl)
            A, B =models_custom.imgconvert(chunk ,l95,DATASET_PATHS,height, width, 0, scaleImg=0) # I am controling through scaleimg
            print(fl)
            with open(fl, 'wb') as handle:            
                    pickle.dump(A, handle, protocol=pickle.HIGHEST_PROTOCOL)
            A=None
            B=None
    except Exception as e:
        print(str(e))
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1,backend='multiprocessing')(delayed(cutpick)(chunk) for chunk in chunks)

In [6]:
import importlib
importlib.reload(models_custom)
# Creates test and train dataset based on
#folder ='three'

#folder ='imgs10'

folder = 'stats'


#folder="three_mfcc"

import pickle5 as pickle
pathG="../data"

fl=pathG+'/w2v2/'+folder
with open(fl+'/8161_39117_765045388.pickle', 'rb') as f:
    ab = pickle.load(f)


chunk_size =8

l95=5000 # Do not increase more than 5000
width=ab.shape[1]
height=ab.shape[2]
flt= ab.shape[3]
print(width, height,flt)
X= df
y= df['emotion']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)
#X_test=X_val
#y_test=y_val

#print(fl)

val_x,y_val_wav = models_custom.getarr(X_val,width, height, flt,fl)
test_x,y_test_wav = models_custom.getarr(X_test,width, height, flt,fl)


import pickle5 as pickle

ab= (X_train,val_x, y_val_wav, test_x, y_test_wav)
fl0=fl+"/batches/xvaltest.pickle"
with open(fl0,"wb") as f:
    pickle.dump(ab, f, protocol=pickle.HIGHEST_PROTOCOL)
f.close()
#print(val_x.shape)
chunks=models_custom.split_dataframe(X_train,chunk_size)
k=0
for chunk in chunks[0:len(chunks)]:
    if chunk.shape[0]>5:
        print(chunk.shape[0])
        fl1=fl+"/batches/xtrain"+str(k)+".pickle"
        ab=models_custom.getarr(chunk,width, height, flt,fl)
        with open(fl1,"wb") as f:
            pickle.dump(ab, f, protocol=pickle.HIGHEST_PROTOCOL)
        k=k+1


5010 9 1
925 (2, 5010, 9, 1) svmem(total=105487597568, available=102708158464, percent=2.6, used=1834610688, free=100313468928, active=1644634112, inactive=3080593408, buffers=81969152, cached=3257548800, shared=4300800, slab=215818240)
847 (3, 5010, 9, 1) svmem(total=105487597568, available=102708158464, percent=2.6, used=1834610688, free=100313468928, active=1644634112, inactive=3080593408, buffers=81969152, cached=3257548800, shared=4300800, slab=215818240)
254 (4, 5010, 9, 1) svmem(total=105487597568, available=102708158464, percent=2.6, used=1834610688, free=100313468928, active=1644634112, inactive=3080593408, buffers=81969152, cached=3257548800, shared=4300800, slab=215818240)
479 (5, 5010, 9, 1) svmem(total=105487597568, available=102708146176, percent=2.6, used=1834622976, free=100313468928, active=1644634112, inactive=3080581120, buffers=81969152, cached=3257536512, shared=4300800, slab=215818240)
199 (6, 5010, 9, 1) svmem(total=105487597568, available=102708146176, percent=2